In [2]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Load the ESG data (replace with your file path)
file_path = r"C:/Users/rajas/sp500_esg_data.xlsx"
esg_data = pd.read_excel(file_path, sheet_name='sp500_esg_data')

# Select governance-related columns: 'governanceScore', 'CEO Tenure', '% Indep Dir on Board' (if available)
# Assuming we only have 'governanceScore' from this dataset
governance_data = esg_data[['governanceScore']].dropna()

# Standardize the data for PCA
scaler = StandardScaler()
governance_data_scaled = scaler.fit_transform(governance_data)

# Apply PCA
pca = PCA(n_components=1)  # We only have one governance variable here
pca_result = pca.fit_transform(governance_data_scaled)

# Create a DataFrame with the PCA results
pca_df = pd.DataFrame(pca_result, columns=['Principal Component 1'])

# Concatenate PCA results with the original company symbols for easier interpretation
pca_merged_data = pd.concat([esg_data[['Symbol']], pca_df], axis=1)

# Display the results
print(pca_merged_data.head())

# Strategy: Long stocks with high PC1 (strong governance), Short stocks with low PC1 (weak governance)
long_positions = pca_merged_data[pca_merged_data['Principal Component 1'] > 0]
short_positions = pca_merged_data[pca_merged_data['Principal Component 1'] < 0]

print("\nLong Positions (High Governance Scores):")
print(long_positions)

print("\nShort Positions (Low Governance Scores):")
print(short_positions)


  Symbol  Principal Component 1
0      A              -0.277875
1    AAL              -0.897421
2   AAPL               1.234002
3   ABBV               1.691726
4    ABT               0.753160

Long Positions (High Governance Scores):
    Symbol  Principal Component 1
2     AAPL               1.234002
3     ABBV               1.691726
4      ABT               0.753160
5     ACGL               1.936770
8      ADI               0.128991
..     ...                    ...
417    WTW               1.876665
419   WYNN               1.793443
421    XOM               0.822512
424    ZBH               0.549727
425    ZTS               0.933476

[166 rows x 2 columns]

Short Positions (Low Governance Scores):
    Symbol  Principal Component 1
0        A              -0.277875
1      AAL              -0.897421
6      ACN              -1.059243
7     ADBE              -0.717106
9      ADM              -0.781834
..     ...                    ...
414    WMB              -1.729647
418     WY          

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

# Define the start and end dates
start_date = "2015-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# List of selected stocks
selected_stocks = ['AACG', 'AADI', 'AADR', 'AAL', 'AAME', 'ZZZ', 'ZYXI', 'ZYME', 'ZVSA', 'ZVRA', 'ZURA']

# Function to safely download stock data and handle errors
def safe_download(symbol, start, end):
    try:
        data = yf.download(symbol, start=start, end=end)
        return data
    except Exception as e:
        print(f"Unable to import {symbol}: {str(e)}")
        return None

# Download stock data
stock_data = {symbol: safe_download(symbol, start_date, end_date) for symbol in selected_stocks}

# Remove any stocks that failed to download
valid_stocks = [symbol for symbol, data in stock_data.items() if data is not None]
stock_data = {symbol: data for symbol, data in stock_data.items() if data is not None}

# Calculate monthly returns
monthly_returns = {symbol: data['Adj Close'].resample('M').last().pct_change() 
                   for symbol, data in stock_data.items()}

# Combine all returns into a single DataFrame
all_returns = pd.DataFrame({symbol: returns for symbol, returns in monthly_returns.items()})

# Implement the backtesting strategy
n_long = min(5, len(valid_stocks))
n_short = min(10, len(valid_stocks) - n_long)
long_weights = [1/n_long] * n_long
short_weights = [-1/n_short] * n_short
weights = long_weights + short_weights

# Calculate portfolio returns
portfolio_returns = all_returns.iloc[:, :len(weights)].mul(weights).sum(axis=1)

# Calculate cumulative returns
cumulative_returns = (1 + portfolio_returns).cumprod()

# Calculate annualized return
annualized_return = (cumulative_returns.iloc[-1] ** (1 / (len(cumulative_returns) / 12))) - 1

# Calculate Sharpe ratio (assuming risk-free rate of 0)
sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(12)

# Create an interactive plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=cumulative_returns.index,
    y=cumulative_returns.values,
    mode='lines',
    name='Cumulative Returns'
))

fig.update_layout(
    title='Cumulative Returns of Alphabetical Strategy',
    xaxis_title='Date',
    yaxis_title='Cumulative Returns',
    hovermode='x unified'
)

# Add annotations for key metrics
fig.add_annotation(
    x=1, y=1, xref='paper', yref='paper',
    text=f'Annualized Return: {annualized_return:.2%}<br>Sharpe Ratio: {sharpe_ratio:.2f}',
    showarrow=False,
    font=dict(size=12),
    align='right',
    xanchor='right',
    yanchor='top'
)

# Show the interactive plot
fig.show()

# Print the results
print(f"Cumulative Returns: {cumulative_returns.iloc[-1]:.2f}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Cumulative Returns: 0.80
Annualized Return: -2.24%
Sharpe Ratio: 0.14


In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Define the date range
start_date = "2015-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Function to get S&P 500 symbols
def get_sp500_symbols():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url)[0]
    return table['Symbol'].tolist()

# Get S&P 500 symbols
sp500_symbols = get_sp500_symbols()

# Sort symbols alphabetically
sp500_symbols.sort()

# Select top 5 and bottom 10 stocks
top_5 = sp500_symbols[:5]
bottom_10 = sp500_symbols[-10:]

# Combine selected stocks
selected_stocks = top_5 + bottom_10

# Download stock data
data = yf.download(selected_stocks, start=start_date, end=end_date)['Adj Close']

# Calculate monthly returns
monthly_returns = data.resample('M').last().pct_change()

# Create portfolio weights
weights = [0.2] * 5 + [-0.1] * 10  # 20% for each top stock, -10% for each bottom stock

# Calculate portfolio returns
portfolio_returns = monthly_returns.mul(weights).sum(axis=1)

# Calculate cumulative returns
cumulative_returns = (1 + portfolio_returns).cumprod()

# Calculate performance metrics
total_return = cumulative_returns.iloc[-1] - 1
annualized_return = (1 + total_return) ** (12 / len(cumulative_returns)) - 1
sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(12)

# Create interactive plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=cumulative_returns.index,
    y=cumulative_returns.values,
    mode='lines',
    name='Cumulative Returns'
))

fig.update_layout(
    title='Alphabetical Order Strategy: Cumulative Returns',
    xaxis_title='Date',
    yaxis_title='Cumulative Returns',
    hovermode='x unified'
)

fig.add_annotation(
    x=1, y=1, xref='paper', yref='paper',
    text=f'Total Return: {total_return:.2%}<br>Annualized Return: {annualized_return:.2%}<br>Sharpe Ratio: {sharpe_ratio:.2f}',
    showarrow=False,
    font=dict(size=12),
    align='right',
    xanchor='right',
    yanchor='top'
)

fig.show()

print(f"Total Return: {total_return:.2%}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

[*********************100%***********************]  15 of 15 completed


Total Return: 24.29%
Annualized Return: 2.24%
Sharpe Ratio: 0.24
